In [112]:
import pandas as pd
import numpy as np

In [113]:
df1 = pd.read_csv("important_polls1.csv")
df2 = pd.read_csv("important_polls2.csv")
df3 = pd.read_csv("important_polls3.csv")
df1["src"] = "vso_1"
df2["src"] = "public180811191"
df3["src"] = "2oprosa"
df = pd.concat([df1, df2, df3])
df = df.reset_index(drop=True)

In [114]:
df = df.fillna("")

In [115]:
ids = (
    317070450,
    306376002,
    307868525,
    306163311,
    334522121,
    335862548,
    377946016,
    704161046,
    178702275,
    299523767,
)
df.loc[df.id.isin(ids), "text"] = ""

In [116]:
df.loc[df.text.str.contains("club"), "text"] = ""
df.loc[
    df.text.str.contains("Как на счет шикарного портрета для любимой мамы?"),
    "text",
] = ""
df.loc[
    df.text.str.contains(
        "Вам не нужно теперь ломать голову, что подарить на праздник. "
    ),
    "text",
] = ""

In [117]:
df.loc[df.id == 332110717, "text"] = df.loc[df.id == 332110717, "text"].str.replace(
    "Навеяно комментариями под этим опросом:\nhttps://vk.com/wall-84926122_1489624\n\n",
    "",
)
df.loc[df.id == 325960904, "text"] = df.loc[
    df.id == 325960904, "text"
].str.replace(
    ", основанном на этом опросе: https://vk.com/wall-84926122_1427610\n",
    ".",
)
df.loc[df.id == 341547905, "text"] = df.loc[
    df.id == 341547905, "text"
].str.replace(
    "Опрос вдохновленный https://vk.com/vso_1?w=wall-84926122_1631418\n",
    "",
)
df.loc[df.id == 335862548, "text"] = df.loc[
    df.id == 335862548, "text"
].str.replace(
    "Опрос вдохновленный https://vk.com/vso_1?w=wall-84926122_1631418\n",
    "",
)

df.loc[df.id == 805488215, "text"] = ""
df.loc[df.id == 805488215, "answer1"] = "Вы получаете любую вещь/услугу (за исключением просто бренда/компании и тд) представленную в рекламе, которую вы видите. \nМатериализуя это в любом удобном для вас месте."
df.loc[df.id == 805488215, "answer2"] = "Вы получаете 10 000 000$ , у налоговой нет к вам претензий"

df.loc[df.id == 791760680, "text"] = (
    "Вас не устраивает современная политическая ситуация. Вы решаетесь бороться с этим, и финальный шаг - выбор"
)
df.loc[df.id == 791760680, "answer1"] = (
    "Воскресить одного случайного политика. Он появится в своей стране/её правоприемнике, устроит государственный переворот (не без жертв среди гражданского населения) и объявит себя главой государства. Политик, как и страна, случайны — это может быть и в глубинах Африки, и в Соединённых Штатах, где угодно."
)
df.loc[df.id == 791760680, "answer2"] = (
    "Убить одного случайного политика. Его должность займёт его конкурент с абсолютно противоположными взглядами. Опять же, политик, как и страна, абсолютно случайны."
)

In [118]:
df = df.drop(df[df.text.str.contains("Wild Politics")].index)
df = df.drop(df[df.text.str.contains("https://vk.com/")].index)

In [119]:
bad_samples = [319068439, 305894804, 357360639, 704933961, 353187584]
df = df.drop(df[df.id.isin(bad_samples)].index)

In [120]:
import Levenshtein

def remove_spam(text):
    if Levenshtein.distance(text.lower(), "что бы вы выбрали?") <= 8:
        return ""
    if Levenshtein.distance(text.lower(), "что бы ты выбрал?") <= 8:
        return ""
    if Levenshtein.distance(text.lower(), "как бы вы поступили?") <= 8:
        return ""  
    if Levenshtein.distance(text.lower(), "что вы скажете?") <= 8:
        return ""  
    return text

df.question = df.question.apply(remove_spam)
df.text = df.text.apply(remove_spam)

In [121]:
cond1 = df.answer1.str.startswith("1)") & df.answer2.str.startswith("2)")
cond2 = df.answer1.str.startswith("1.") & df.answer2.str.startswith("2.")
cond = cond1 | cond2
df.loc[cond, "answer1"] = df.loc[cond, "answer1"].str.slice(2)
df.loc[cond, "answer2"] = df.loc[cond, "answer2"].str.slice(2)

In [124]:
import re

def preprocess(text):
    text = (
        text.replace("ё", "е")
        .replace("#", "*")
        .replace(" ,", ", ")
        .replace("( ", "(")
        .replace(" ?", "?")
        .replace(" )", ")")
        .replace("\\", "/")
        .replace("\n", " ")
        .replace("–", "-")
        .replace("—", "-")
        .replace("&quot;", "")
    )
    pattern = r"[^а-яА-Яa-zA-Z\s\d\$\(\)?!,\.%/+\-:;*]"
    text = re.sub(pattern, "", text).strip()
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"_+", "_", text)
    text = re.sub(r",(?![\s\d])", ", ", text)
    pattern = r"https?://[^\s]+"
    text = re.sub(pattern, "https://t.me/polls4life", text)
    return text.strip()

preprocess("ghd1342bt wcf-w-\n eq]cwefn$?(!),.%/;;;;::🤕ת בוחר:+-***")

'ghd1342bt wcf-w- eqcwefn$?(!), .%/;;;;:: :+-***'

In [125]:
df.text = df.text.apply(preprocess)
df.question = df.question.apply(preprocess)
df.answer1 = df.answer1.apply(preprocess)
df.answer2 = df.answer2.apply(preprocess)

df.text = df.text.apply(preprocess)
df.question = df.question.apply(preprocess)
df.answer1 = df.answer1.apply(preprocess)
df.answer2 = df.answer2.apply(preprocess)

In [126]:
df.loc[df.question=="?", "question"] = ""

In [127]:
df.loc[df.text.str.lower() == df.question.str.lower(), "question"] = ""

In [128]:
def levenshtein_distance(a, b):
    return Levenshtein.distance(a, b)


# применяем функцию к столбцам text и question
df["distance"] = df.apply(
    lambda row: levenshtein_distance(row["text"], row["question"]), axis=1
)

# заменяем поле text на "" если расстояние меньше 3
df.loc[df["distance"] < 3, "text"] = ""

# удаляем столбец distance
df.drop("distance", axis=1, inplace=True)

In [129]:
df.question = df.text + " " + df.question
del df["text"]

In [130]:
df.loc[len(df)]

id                                                                                                                                                                                                               262900654
question                                                                                                                                                                                       Что бы вы выбрали и почему?
answer1     На всех важных событиях вашей жизни, будь то день рождения, бизнес встреча, конференция и тд. заказывать и пить горячий капучино с пакетиком чая и всем настойчиво рекомендовать попробовать ваш напиток богов
answer2                                                                   Всегда чистить зубы только пальцем (кстати, попробуйте, это не так просто как кажется) или вы можете просить другого человека почистить вам зубы
rate1                                                                                                                       

In [131]:
pd.set_option("display.max_colwidth", None)

In [132]:
df = df.drop_duplicates()

In [133]:
df

,id,question,answer1,answer2,rate1,rate2,votes1,votes2,src
0,885598345,Хотите ли вы детей?,Да,Нет,51.74,48.26,6926,6461,vso_1
1,966697749,"Вы коллекционер монет. Однажды на барахолке покупаете у незнакомого продавца монету, представляющую исключительную ценность и которую практически невозможно найти в продаже. Проведя экспертизу, обнаруживаете, что приобрели за огромную сумму мастерски исполненную подделку. А продавца того больше не встретите. У вас есть старый друг, с которым вы с детства вместе увлекаетесь нумизматикой. Увидев монету, он просит вас перепродать ее ему, уверяя, что давно о ней мечатет и предлагает даже большую цену, чем вы за нее отдали.",Продать монету. Друг будет всю жизнь считать монету подлинной и любоваться ею долгими зимними вечерами,"Признаться, что монета поддельная",38.45,61.55,809,1295,vso_1
2,967001095,"Вы изобрели революционное противозачаточное. Вкратце, она работает так: женщина проглатывает невидимую невооруженным глазом пилюлю, та добирается до матки и разворачивается в небольшую медную спираль. Внешне спираль незаметна, не вызывает дискомфорта и безвредна для здоровья самок мышей и обезьян, на которых ее проверяли. Спирали хватит на 25-30 лет непрерывного использования. Выключается (сворачивается обратно в пилюлю) она пультом. Теперь нужно проверить пилюлю на людях - и желательно до конца года, т.к. в январе грантодатель потребует отчета за выделенные средства. Если не успеете, вполне возможно, что денег Вам больше не дадут, и проект придется свернуть. А отыскать добровольцев не так-то просто: к власти этой весной пришли консервативные силы, считающие разработку противозачаточных пустой тратой денег. В тех нескольких больницах, где Вашу идею приняли позитивно, тестирование пилюли можно начать не раньше сентября, а развернуть в полном масштабе - с октября. В пятницу в баре Вы разговорились с одной чернокожей дамой. Она представилась руководителем отдела семейного благополучия благотворительного фонда им. Марии Терезы. Она жаловалась, что все их усилия вытащить подопечных из нищеты тщетны, в том числе, из-за неконтролируемой рождаемости последних. Она предложила сотрудничество: Вы предоставляете Фонду пилюли и пульт, а она и две доверенные сотрудницы помещают при Вас каждую пилюлю в бутылку минералки и предлагают бутылки пациенткам аффилированных с Фондом лечебниц. Она обещает, что: - воду с пилюлей будут давать небеременным пациенткам, у которых больше 3 детей, - данные о здоровье подопытных, выгруженные из общих баз лечебниц Фонда, присылать еженедельно в анонимизированном виде через защищенный мессенджер, - никто, кроме них троих, знать о Вашем сотрудничестве не будет, - при возникновении жалоб на сложности с зачатием, спираль будет оперативно отключена. Вам удалось проверить ее слова. Похоже, она говорит правду: Фонд действительно существует, работает в беднейших странах Африки, фактически, заменяя там государственных врачей-терапевтов, и она занимает в нем небольшой руководящий пост.","Согласиться. Дама внушает доверие, но неизвестно, насколько на практике будут соблюдаться достигнутые договоренности. А лично контролировать - в Африку не наездишься...","Отказаться и продолжить искать подопытных самому. Неизвестно, удастся ли набрать достаточное количество до конца декабря.",86.80,13.20,848,129,vso_1
3,966790489,"Представим, что вас крупно подставили таким образом, что вы должны выплатить мошеннику один миллион рублей. Вы можете подать в суд и доказать, что вы ничего не должны, но за весь процесс вы потратите полтора миллиона рублей. Компенсацию получить, увы, не получится, но вы ничего не заплатите тому, кто вас подставил, а сам он получит наказание в виде исправительных работ.",Отдать мошеннику миллион рублей.,"Не давать мошеннику денег, но потратить на суд полтора миллиона рублей.",28.62,71.38,417,1040,vso_1
4,965896595,"Вам заплатят 1 млн$ если вы переночуете в 100 самых мистических местах в мире. Это замки, острова, дома с привидениями, заброшенные бункеры и т.п. Возить по этим местам 

In [134]:
df.to_csv("important_polls_cleared.csv", index=False)